In [3]:
%pip install numpy pandas seaborn matplotlib plotly scikit-learn nltk wordcloud

   ---------------------------------------- 0.0/299.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/299.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/299.9 kB ? eta -:--:--
   -- ------------------------------------- 20.5/299.9 kB 108.9 kB/s eta 0:00:03
   ---- ----------------------------------- 30.7/299.9 kB 163.8 kB/s eta 0:00:02
   ---- ----------------------------------- 30.7/299.9 kB 163.8 kB/s eta 0:00:02
   ----- ---------------------------------- 41.0/299.9 kB 151.3 kB/s eta 0:00:02
   ----- ---------------------------------- 41.0/299.9 kB 151.3 kB/s eta 0:00:02
   ------------ --------------------------- 92.2/299.9 kB 261.7 kB/s eta 0:00:01
   ------------ --------------------------- 92.2/299.9 kB 261.7 kB/s eta 0:00:01
   ------------- -------------------------- 102.4/299.9 kB 227.0 kB/s eta 0:00:01
   ------------- -------------------------- 102.4/299.9 kB 227.0 kB/s eta 0:00:01
   ---------------- ----------------------- 122.9/299.